In [ ]:
#importing relevant modules
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
from matplotlib.animation import FFMpegWriter
from mpl_toolkits.basemap import Basemap
import os
import requests
import dateutil.parser
from datetime import timedelta, datetime
from obspy import UTCDateTime

In [ ]:
firstquakeid = "2019p738432" # user input Geonet public ID 

In [ ]:
#Queries Geonet API to retreive mainshock's data
quake = requests.get("https://api.geonet.org.nz/quake/"+ firstquakeid).json()
#Origin time from json file, in datetime object
starttime = dateutil.parser.isoparse(quake['features'][0]['properties']['time'])
endtime = starttime + timedelta(days = 1) #end time, one day after

start, end = starttime.strftime('%Y-%m-%dT%H:%M:%S'), endtime.strftime('%Y-%m-%dT%H:%M:%S')
lon, lat = quake['features'][0]['geometry']['coordinates'][0], quake['features'][0]['geometry']['coordinates'][1]
latlon = str(lon) + "+" + str(lat)

In [ ]:
quakejson = requests.get(
        "http://wfs.geonet.org.nz/geonet/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=geonet:quake_search_v1&outputFormat=json&cql_filter=origintime>=" 
        + start + "+AND+origintime<=" + end + "+AND+DWITHIN(origin_geom,Point+(" + latlon + "),50000,meters)").json()

In [ ]:
quakejson['features'][0]
lat, lon, mag, time = ([] for i in range(4))
for a in range(len(quakejson['features'])):
    lon.append(quakejson['features'][a]['geometry']['coordinates'][0])
    lat.append(quakejson['features'][a]['geometry']['coordinates'][1])
    mag.append(quakejson['features'][a]['properties']['magnitude'])
    time.append(UTCDateTime(quakejson['features'][a]['properties']['origintime']))
df = pd.DataFrame({"lat":lat, "lon":lon, "time":time,"mag":mag})
df = df.sort_values(by="time")

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10), dpi = 100)
ax=fig.add_axes([0,0,1,1])
m = Basemap(projection = "merc", llcrnrlon = lon[0] - .25, llcrnrlat =  lat[0] - .25,
              urcrnrlon =  lon[0] + .25, urcrnrlat = lat[0] + .25, epsg = 2193, fix_aspect=False)
m.arcgisimage(service = "World_Topo_Map", xpixels = 1000, ypixels = 1000, dpi = 1000)
timelist = [list(df['time'])[0] + 60 * x for x in range(30)]
stime = [str(UTCDateTime(start)+ 60 * i) for i in list(range(len(timelist)))]
roundmag = [(int(str(x)[0])+1)**3 * 2 for x in mag]
scat= m.scatter([],[], alpha = 0.6, edgecolor= "black", linewidths = 0.6, c= 'crimson')  
l2 = plt.scatter([],[], s = 2**3.2*2, edgecolors='none', c= 'crimson')
l3 = plt.scatter([],[], s = 3**3.2*2, edgecolors='none', c= 'crimson')
l4 = plt.scatter([],[], s = 4**3.2*2, edgecolors='none', c= 'crimson')
l5 = plt.scatter([],[], s = 5**3.2*2, edgecolors='none', c= 'crimson')
l6 = plt.scatter([],[], s = 6**3.2*2, edgecolors='none', c= 'crimson')
l7 = plt.scatter([],[], s = 7**3.2*2, edgecolors='none', c= 'crimson')
l8 = plt.scatter([],[], s = 8**3.2*2, edgecolors='none', c= 'crimson')
labels = [str(x + 1) for x in range(7)]
annotation = ax.annotate(str(stime[0]), xy=(625, 950), xycoords='axes pixels',  fontsize=14, c ='black')
annotationcount = ax.annotate("Earthquake count: 0", xy = (20, 950), xycoords='axes pixels', fontsize = 17, c = 'black')
annotation.set_animated(True)
annotationcount.set_animated(True)
ax.set_aspect('auto')
plt.legend([l2,l3,l4,l5,l6,l7, l8], labels, title = "Magnitude", loc = 'lower center', labelspacing = 1, ncol = 7, borderpad = 1.7,
          columnspacing = 2.3, handletextpad = 1.2)
def init():
    scat.set_offsets(np.c_[-1000,-1000])
    return scat, annotation, annotationcount
def update(a):
    df2 = df[df['time'] < timelist[a]]
    x, y = m(list(df2['lon']), list(df2['lat']))
    scat.set_offsets(np.c_[x, y])
    scat._sizes=[(int(str(x)[0])+1)**3.2 * 2 for x in list(df2['mag'])]
    annotation.set_text(stime[a])
    annotationcount.set_text("Earthquake count: " + str(len(df2)))
    return scat,
ani = animation.FuncAnimation(fig, update, init_func=init, blit=False, frames = len(timelist))
writer = FFMpegWriter(fps=10, bitrate=1800)
ani.save((firstquakeid + ".mp4"), writer=writer)